In [1]:
from SmartApi import SmartConnect
import pyotp
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
from logzero import logger
import pandas as pd
import requests
from datetime import datetime, date, time, timedelta
import config
import math


In [8]:
url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
d = requests.get(url).json()
global token_df
token_df = pd.DataFrame.from_dict(d)
token_df['expiry'] = pd.to_datetime(token_df['expiry'])
token_df = token_df.astype({'strike': float})
token_df

/tmp/ipykernel_60324/1067504763.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  token_df['expiry'] = pd.to_datetime(token_df['expiry'])


,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
0,20171,77GA33B-SG,77GA33B,NaT,-1.0,100,,NSE,1.000000
1,20243,767NL33B-SG,767NL33B,NaT,-1.0,100,,NSE,1.000000
2,16725,RPGLIFE-EQ,RPGLIFE,NaT,-1.0,1,,NSE,5.000000
3,2916,686JH36-SG,686JH36,NaT,-1.0,100,,NSE,1.000000
4,15402,EIHAHOTELS-BL,EIHAHOTELS,NaT,-1.0,1,,NSE,5.000000
...,...,...,...,...,...,...,...,...,...
181932,828611,TITAN241113420PE,TITAN,2024-01-11,342000.0,175,OPTSTK,BFO,5.000000
181933,1106473,UBL242081700PE,UBL,2024-02-08,170000.0,400,OPTSTK,BFO,5.000000
181934,833586,MGL24111950PE,MGL,2024-01-11,95000.0,800,OPTSTK,BFO,5.000000
181935,2096545,BALRAMCHIN24FEB370CE,BALRAMCHIN,2024-02-29,37000.0,1600,OPTSTK,BFO,5.000000


In [2]:
obj = SmartConnect(api_key=config.API_KEY)
data = obj.generateSession(config.USER_NAME,config.PIN,pyotp.TOTP(config.TOKEN).now())
data

# AUTH_TOKEN = data['data']['jwtToken']
# refreshToken = data['data']['refreshToken']
# FEED_TOKEN = obj.getfeedToken()
# res = obj.getProfile(refreshToken)
# refreshToken = data['data']['refreshToken']
# userProfile = obj.getProfile(refreshToken)
# print(f'Angel account logged in {obj.rmsLimit()}')

ReadTimeout: HTTPSConnectionPool(host='apiconnect.angelbroking.com', port=443): Read timed out. (read timeout=7)

In [ ]:
def AngelLogin(apikey, username, pwd, totptoken):
    obj = SmartConnect(api_key=apikey)
    data = obj.generateSession(username,pwd,pyotp.TOTP(totptoken).now())
    refreshToken = data['data']['refreshToken']
    userProfile = obj.getProfile(refreshToken)
    print(f'Angel account logged in {obj.rmsLimit()}')
    return obj

In [ ]:
def getTokenInfo (exch_seg, instrumenttype,symbol,strike_price,pe_ce):
    df = config.TOKEN_MAP
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') & (df['symbol'].str.contains('EQ')) ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

    

    

In [ ]:
def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})
    config.TOKEN_MAP = token_df


In [10]:
AUTH_TOKEN = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']
FEED_TOKEN = obj.getfeedToken()
res = obj.getProfile(refreshToken)

In [ ]:
url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
d = requests.get(url).json()
token_df = pd.DataFrame.from_dict(d)
token_df['expiry'] = pd.to_datetime(token_df['expiry']).apply(lambda x: x.date())
token_df = token_df.astype({'strike': float})

# config.TOKEN_MAP = token_df

In [ ]:
#for nse
symbol_token = token_df[(token_df.exch_seg == 'NFO') & (token_df.instrumenttype == 'OPTIDX') & (token_df.name == 'NIFTY')]
symbol_token['expiry'] = pd.to_datetime(symbol_token['expiry'])
symbol_token['lotsize'] = pd.to_numeric(symbol_token['lotsize'])
symbol_token

In [ ]:
nse_df = symbol_token[(symbol_token['expiry']=='2024-01-11')]
nse_df
nse_df['strike'] = nse_df['strike'] / 100
stream_symbols = nse_df[(nse_df['strike'] >= 21500) & (nse_df['strike'] <= 21700)]
stream_symbols

In [11]:
symbol_token = token_df[(token_df.exch_seg == 'MCX') & (token_df.instrumenttype == 'OPTFUT') & ((token_df.name.str.contains('CRUDE')))]
symbol_token['expiry'] = pd.to_datetime(symbol_token['expiry'])
symbol_token['lotsize'] = pd.to_numeric(symbol_token['lotsize'])

/tmp/ipykernel_60324/2658722518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symbol_token['expiry'] = pd.to_datetime(symbol_token['expiry'])
/tmp/ipykernel_60324/2658722518.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symbol_token['lotsize'] = pd.to_numeric(symbol_token['lotsize'])


In [ ]:
# token_df['instumenttype'].unique()

In [12]:
mcx_crude = symbol_token[(symbol_token['expiry']=='2024-01-17')]
mcx_crude['strike'] = mcx_crude['strike'] / mcx_crude['lotsize']
stream_symbols = mcx_crude[(mcx_crude['strike'] >= 5900) & (mcx_crude['strike'] <= 6200)]
stream_symbols

/tmp/ipykernel_60324/3490757104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcx_crude['strike'] = mcx_crude['strike'] / mcx_crude['lotsize']


,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size
94432,425303,CRUDEOIL24JAN6000CE,CRUDEOIL,2024-01-17,6000.0,100,OPTFUT,MCX,10.000000
94751,425304,CRUDEOIL24JAN5950CE,CRUDEOIL,2024-01-17,5950.0,100,OPTFUT,MCX,10.000000
94785,425355,CRUDEOIL24JAN5900PE,CRUDEOIL,2024-01-17,5900.0,100,OPTFUT,MCX,10.000000
95064,425299,CRUDEOIL24JAN6200CE,CRUDEOIL,2024-01-17,6200.0,100,OPTFUT,MCX,10.000000
95108,425301,CRUDEOIL24JAN6100CE,CRUDEOIL,2024-01-17,6100.0,100,OPTFUT,MCX,10.000000
95298,425350,CRUDEOIL24JAN6150PE,CRUDEOIL,2024-01-17,6150.0,100,OPTFUT,MCX,10.000000
95977,425300,CRUDEOIL24JAN6150CE,CRUDEOIL,2024-01-17,6150.0,100,OPTFUT,MCX,10.000000
96029,425349,CRUDEOIL24JAN6200PE,CRUDEOIL,2024-01-17,6200.0,100,OPTFUT,MCX,10.000000
96635,425302,CRUDEOIL24JAN6050CE,CRUDEOIL,2024-01-17,6050.0,100,OPTFUT,MCX,10.000000
96902,425305,CRUDEOIL24JAN5900CE,CRUDEOIL,2024-01-17,5900.0,100,OPTFUT,MCX,10.000000


In [13]:
correlation_id = "test123"
action = 1
mode = 3
token_list = [
    {
        "exchangeType": 5,
        "tokens": ["425353","425303"] # 6000 put # 21700 PE, 21650CE
    }
]
#retry_strategy=0 for simple retry mechanism
sws = SmartWebSocketV2(AUTH_TOKEN, config.API_KEY, config.USER_NAME, FEED_TOKEN)

#retry_strategy=1 for exponential retry mechanism
# sws = SmartWebSocketV2(AUTH_TOKEN, API_KEY, CLIENT_CODE, FEED_TOKEN,max_retry_attempt=3, retry_strategy=1, retry_delay=10,retry_multiplier=2, retry_duration=30)

def on_data(wsapp, message):
    logger.info("Ticks: {}".format(message))
    
    try:
        config.LIVE_FEED_JSON[message['token']] = {'token' : message['token'], 
                                                   'timestamp': message['exchange_timestamp'],
                                                    'LTP': message['last_traded_price'],
                                                    'openinterest': message['open_interest'],
                                                    'tbq': message['total_buy_quantity'],
                                                    'tsq': message['total_sell_quantity']
                                                }
    except Exception as e:
        print(e)
    # close_connection()

def on_control_message(wsapp, message):
    logger.info(f"Control Message: {message}")

def on_open(wsapp):
    logger.info("on open")
    some_error_condition = False
    if some_error_condition:
        error_message = "Simulated error"
        if hasattr(wsapp, 'on_error'):
            wsapp.on_error("Custom Error Type", error_message)
    else:
        sws.subscribe(correlation_id, mode, token_list)
        # sws.unsubscribe(correlation_id, mode, token_list1)

def on_error(wsapp, error):
    logger.error(error)

def on_close(wsapp):
    logger.info("Close")

def close_connection():
    sws.close_connection()


# Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error
sws.on_close = on_close
sws.on_control_message = on_control_message

sws.connect()

[I 240111 22:51:26 764574032:35] on open
[W 240111 22:51:47 smartWebSocketV2:319] Attempting to resubscribe/reconnect (Attempt 1)...
[I 240111 22:51:57 764574032:35] on open


In [ ]:
# TODOS:
# 1. function to get nearest symbols of the strike price to the spot price
